https://wiki.thebiogrid.org/doku.php/biogrid_tab_version_2.0
http://pages.cs.wisc.edu/~legault/writeup-776.pdf

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.multitest as sm

In [2]:
# data = pd.read_table("BIOGRID-ORGANISM-Homo_sapiens-3.5.182.tab2.txt", low_memory = False)
data = pd.read_table("BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.182.tab2.txt", low_memory = False)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [3]:
data.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,68770,851136,854020,31676,34272,YLR418C,YOL145C,CDC73,CTR9,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
1,68771,854020,851136,34272,31676,YOL145C,YLR418C,CTR9,CDC73,CDP1|L000003477,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
2,68774,851136,854290,31676,34518,YLR418C,YOR123C,CDC73,LEO1,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
3,68775,854290,851136,34518,31676,YOR123C,YLR418C,LEO1,CDC73,L000000936,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
4,68778,851136,852582,31676,32973,YLR418C,YBR279W,CDC73,PAF1,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID


In [22]:
val = data.copy()
val.set_index("Experimental System Type", inplace = True)
val = val.filter(items = ["Official Symbol Interactor A", "Official Symbol Interactor B", "Pubmed ID"])
val = val.filter(like = "physical", axis = 0)
# val = val.filter(like = "genetic", axis = 0)
print(val.size)
val.head()

522792


,Official Symbol Interactor A,Official Symbol Interactor B,Pubmed ID
Experimental System Type,,,
physical,CDC73,CTR9,14759368
physical,CTR9,CDC73,14759368
physical,CDC73,LEO1,14759368
physical,LEO1,CDC73,14759368
physical,CDC73,PAF1,14759368


In [23]:
graph = nx.from_pandas_edgelist(val, "Official Symbol Interactor A", "Official Symbol Interactor B", create_using=nx.DiGraph)

In [24]:
# nx.draw_networkx(graph)

In [25]:
adj = nx.to_pandas_adjacency(graph)

In [26]:
adj

,CDC73,CTR9,LEO1,PAF1,POB3,RTF1,SPT16,CHD1,CKA1,CKA2,...,ANXA10,LGALS3BP,GAN,TPM3,RCC2,HDHD1,APP,KIF5B,RUVBL1,PFKFB3
CDC73,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTR9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LEO1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PAF1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
POB3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RTF1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPT16,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHD1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CKA1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CKA2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
